In [1]:
from nustar_pysolar import planning
import astropy.units as u
from astropy.time import Time
from astropy.coordinates import get_body, solar_system_ephemeris, get_body_barycentric, SkyCoord

In [2]:
fname = planning.download_occultation_times(outdir='../data/')
print(fname)

../data/NUSTAR.2017_129.SHADOW_ANALYSIS.txt


In [3]:
tstart = '2017-05-19T01:43:00'
tend = '2017-05-19T11:43:00'
orbits = planning.sunlight_periods(fname, tstart, tend)
solar_system_ephemeris.set('jpl') 

<ScienceState solar_system_ephemeris: 'jpl'>

In [4]:
from skyfield.api import Loader
load = Loader('../data')
ts = load.timescale()
planets = load('jup310.bsp')


[#################################] 100% deltat.data
[#################################] 100% deltat.preds
[#################################] 100% Leap_Second.dat
[#################################] 100% jup310.bsp


# Use the astropy interface to get the location of Jupiter as the time that you want to use.

In [5]:
dt = 0.

# Using JPL Horizons web interface at 2017-05-19T01:34:40
horizon_ephem = SkyCoord(*[193.1535, -4.01689]*u.deg)


for orbit in orbits:
    tstart = orbit[0]
    tend = orbit[1]
    print()
#    print('Orbit duration: ', tstart.isoformat(), tend.isoformat())
    on_time = (tend - tstart).total_seconds()
    
    point_time = tstart + 0.5*(tend - tstart)
    print('Time used for ephemeris: ', point_time.isoformat())
    
    astro_time = Time(point_time)
    solar_system_ephemeris.set('jpl')


    jupiter = get_body('Jupiter', astro_time)
    
    jplephem = SkyCoord(jupiter.ra.deg*u.deg, jupiter.dec.deg*u.deg)
    
    # Switch to the built in ephemris
    solar_system_ephemeris.set('builtin')
    jupiter = get_body('Jupiter', astro_time)
    
    builtin_ephem = SkyCoord(jupiter.ra.deg*u.deg, jupiter.dec.deg*u.deg)
    
    t = ts.from_astropy(astro_time)
    jupiter, earth = planets['jupiter'], planets['earth']
    astrometric = earth.at(t).observe(jupiter)
    ra, dec, distance = astrometric.radec()
    radeg = ra.to(u.deg)
    decdeg = dec.to(u.deg)
    skyfield_ephem = SkyCoord(radeg, decdeg)
    
    
    print()
    print('Horizons offset to jplephem: ', horizon_ephem.separation(jplephem))
    print()
    print('Horizons offset to "built in" ephemeris: ', horizon_ephem.separation(builtin_ephem))
    print()
    print('Horizons offset to Skyfield ephemeris: ', horizon_ephem.separation(skyfield_ephem))

    print()
    break



Time used for ephemeris:  2017-05-19T01:45:00

Horizons offset to jplephem:  0d00m13.0448s

Horizons offset to "built in" ephemeris:  0d00m15.7584s

Horizons offset to Skyfield ephemeris:  0d00m01.6335s



# Conclusion: Use skyfield if you want to reproduce the JPL ephemerides

## Use the jup310.bsp file for Jupiter. Need to confirm which of the avaiable .bsp files are approriate for inner solar system objects as well as the Sun/Moon

In [7]:
dt = 0.

for orbit in orbits:
    tstart = orbit[0]
    tend = orbit[1]
    print()
    on_time = (tend - tstart).total_seconds()
    
    point_time = tstart + 0.5*(tend - tstart)
    print('Time used for ephemeris: ', point_time.isoformat())
    
    astro_time = Time(point_time)
    solar_system_ephemeris.set('jpl')


    jupiter = get_body('Jupiter', astro_time)
    
    jplephem = SkyCoord(jupiter.ra.deg*u.deg, jupiter.dec.deg*u.deg)
    
    # Switch to the built in ephemris
    solar_system_ephemeris.set('builtin')
    jupiter = get_body('Jupiter', astro_time)
    
    builtin_ephem = SkyCoord(jupiter.ra.deg*u.deg, jupiter.dec.deg*u.deg)
    
    t = ts.from_astropy(astro_time)
    jupiter, earth = planets['jupiter'], planets['earth']
    astrometric = earth.at(t).observe(jupiter)
    ra, dec, distance = astrometric.radec()
    radeg = ra.to(u.deg)
    decdeg = dec.to(u.deg)
    skyfield_ephem = SkyCoord(radeg, decdeg)
    
    
    print()
    print('Skyfield offset to jplephem: ', skyfield_ephem.separation(jplephem))
    print()
    print('Skyfield offset to "built in" ephemeris: ', skyfield_ephem.separation(builtin_ephem))

    print()




Time used for ephemeris:  2017-05-19T01:45:00

Skyfield offset to jplephem:  0d00m14.6773s

Skyfield offset to "built in" ephemeris:  0d00m17.3375s


Time used for ephemeris:  2017-05-19T03:21:45

Skyfield offset to jplephem:  0d00m14.6587s

Skyfield offset to "built in" ephemeris:  0d00m17.315s


Time used for ephemeris:  2017-05-19T04:58:25

Skyfield offset to jplephem:  0d00m14.64s

Skyfield offset to "built in" ephemeris:  0d00m17.2925s


Time used for ephemeris:  2017-05-19T06:35:05

Skyfield offset to jplephem:  0d00m14.6213s

Skyfield offset to "built in" ephemeris:  0d00m17.2699s


Time used for ephemeris:  2017-05-19T08:11:50

Skyfield offset to jplephem:  0d00m14.6026s

Skyfield offset to "built in" ephemeris:  0d00m17.2474s


Time used for ephemeris:  2017-05-19T09:48:30

Skyfield offset to jplephem:  0d00m14.584s

Skyfield offset to "built in" ephemeris:  0d00m17.225s



In [6]:
from skyfield.api import Loader, load
load = Loader('../data')
ts = load.timescale()
load.list_bsp()
planets = load('de436.bsp')

Emepheris location for files like "jup*.bsp"
http://naif.jpl.nasa.gov/pub/naif/generic_kernels/spk/satellites/
11-Dec-2013 12:31 jup300.bsp 138M
11-Dec-2013 12:41 jup310.bsp 931M
09-Mar-2015 17:59 jup329.bsp 51M
28-Sep-2012 10:13 mar097.bsp 448M
29-Apr-2015 12:21 nep081.bsp 151M
12-Dec-2013 14:16 nep086.bsp 21M
11-Dec-2013 12:46 nep087.bsp 64M
29-Apr-2015 12:31 nep088.bsp 505M
06-Nov-2015 11:27 plu055.bsp 81M
08-May-2009 08:35 sat319.bsp 52M
11-Aug-2014 08:02 sat368.bsp 82M
24-Jul-2015 16:30 sat375.bsp 280M
11-Oct-2016 14:43 sat393-rocks_pan.bsp 7.0K
11-Oct-2016 14:22 sat393.bsp 586M
11-Oct-2016 14:32 sat393_daphnis.bsp 9.0K
12-Dec-2013 14:48 ura091-rocks-merge.bsp 9.4M
24-Oct-2013 12:12 ura091-rocks.bsp-source 22K
08-Jan-2014 17:29 ura111.bsp 162M
08-Jan-2014 17:31 ura112.bsp 30M

Emepheris location for files like "*.bsp"
ftp://ssd.jpl.nasa.gov/pub/eph/planets/bsp/
Sep 10 2015 TTmTDB.de430.19feb2015.bsp 31.4M
Mar 24 2011 de102.bsp 228.1M
Mar 18 2011 de200.bsp 54.2M
Mar 28 2011 de202.b

In [7]:
print(planets)

SPICE kernel file 'de436.bsp' has 14 segments
  JD 2287184.50 - JD 2688976.50  (1549-12-30 through 2650-01-24)
      0 -> 1    SOLAR SYSTEM BARYCENTER -> MERCURY BARYCENTER
      0 -> 2    SOLAR SYSTEM BARYCENTER -> VENUS BARYCENTER
      0 -> 3    SOLAR SYSTEM BARYCENTER -> EARTH BARYCENTER
      0 -> 4    SOLAR SYSTEM BARYCENTER -> MARS BARYCENTER
      0 -> 5    SOLAR SYSTEM BARYCENTER -> JUPITER BARYCENTER
      0 -> 6    SOLAR SYSTEM BARYCENTER -> SATURN BARYCENTER
      0 -> 7    SOLAR SYSTEM BARYCENTER -> URANUS BARYCENTER
      0 -> 8    SOLAR SYSTEM BARYCENTER -> NEPTUNE BARYCENTER
      0 -> 9    SOLAR SYSTEM BARYCENTER -> PLUTO BARYCENTER
      0 -> 10   SOLAR SYSTEM BARYCENTER -> SUN
      3 -> 301  EARTH BARYCENTER -> MOON
      3 -> 399  EARTH BARYCENTER -> EARTH
      1 -> 199  MERCURY BARYCENTER -> MERCURY
      2 -> 299  VENUS BARYCENTER -> VENUS
